# Ollama PDF RAG Notebook

## Import Libraries


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.chat_models import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [3]:
# Specify the path to your PDF file
loader = PyPDFLoader("D:\MSIS\Gen AI\Pdf_Data_extractor_rag\Data\scammer-agent.pdf")
pages = loader.load()
pages

[Document(metadata={'source': 'D:\\MSIS\\Gen AI\\Pdf_Data_extractor_rag\\Data\\scammer-agent.pdf', 'page': 0}, page_content='Voice-Enabled AI Agents can Perform Common Scams\nRichard Fang\nUIUC\nDylan Bowman\nUIUC\nDaniel Kang\nUIUC\nAbstract\nRecent advances in multi-modal, highly ca-\npable LLMs have enabled voice-enabled AI\nagents. These agents are enabling new applica-\ntions, such as voice-enabled autonomous cus-\ntomer service. However, with all AI capabili-\nties, these new capabilities have the potential\nfor dual use.\nIn this work, we show that voice-enabled AI\nagents can perform the actions necessary to\nperform common scams. To do so, we select a\nlist of common scams collected by the govern-\nment and construct voice-enabled agents with\ndirections to perform these scams. We con-\nduct experiments on our voice-enabled agents\nand show that they can indeed perform the ac-\ntions necessary to autonomously perform such\nscams. Our results raise questions around the\nwidespr

## Split text into chunks

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'source': 'D:\\MSIS\\Gen AI\\Pdf_Data_extractor_rag\\Data\\scammer-agent.pdf', 'page': 0}, page_content='Voice-Enabled AI Agents can Perform Common Scams\nRichard Fang\nUIUC\nDylan Bowman\nUIUC\nDaniel Kang\nUIUC\nAbstract\nRecent advances in multi-modal, highly ca-\npable LLMs have enabled voice-enabled AI\nagents. These agents are enabling new applica-\ntions, such as voice-enabled autonomous cus-\ntomer service. However, with all AI capabili-\nties, these new capabilities have the potential\nfor dual use.\nIn this work, we show that voice-enabled AI\nagents can perform the actions necessary to\nperform common scams. To do so, we select a\nlist of common scams collected by the govern-\nment and construct voice-enabled agents with\ndirections to perform these scams. We con-\nduct experiments on our voice-enabled agents\nand show that they can indeed perform the ac-\ntions necessary to autonomously perform such\nscams. Our results raise questions around the\nwidespr

## Create vector database

In [5]:
# Step 3: Create embeddings using HuggingFace's 
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [6]:
# Step 4: Create Chroma vector store
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [7]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [8]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_store.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [9]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [11]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [12]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be that it discusses the potential risks and vulnerabilities of Large Language Models (LLMs) in various applications, including social engineering scams. The authors suggest that LLMs can be used to autonomously hack websites, exploit zero-day vulnerabilities, and engage in other malicious activities. They also discuss the financial costs of these scams, which can reach up to $40 billion per year.

In [13]:
# Example 2
chat_with_pdf("What is the purpose of the scammer agent?")

The purpose of the scammer agent appears to be simulating a scam by manually talking with a voice agent with a credulous victim, in order to evaluate the success rate of the scams and gather data on how they are performed. The agents were designed to perform various types of scams, including bank transfer scams and credential stealing scams, and were executed multiple times to record their performance metrics, such as overall success rate, number of tool calls, call time, and approximate API cost.

## Clean up (optional)

In [15]:
# Optional: Clean up when done 
vector_store.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
